In [2]:
# Necessary Imports
import numpy as np
import pandas as pd
import ast
import csv
import os
from sklearn.model_selection import StratifiedKFold, KFold, cross_validate

from classification.ElkanotoSVCClassifier import ElkanotoSVCClassifier
from dataPreparation import DataPreparation
from dataPreprocessing import DataPreprocessing

In [1]:
# Global parameters for calculating the data for the csv (and their paths) and for the chosen features
doPreperation = True
doPreprocessing = False
featureFilePath = "genomeFeatures.csv"
genomeFilePath = "genomes.csv"
chosenFeatures = { "kmer": [2, 3, 7], "codon_translation": [], "dicodon_translation" : []}

In [3]:
# Data Preparation of Training Data
if doPreperation:
    dataPreperator = DataPreparation.DataPreparation()
    trainingGenomes = dataPreperator.loadTrainingGenomeSequences()
    with open(genomeFilePath, mode="w", newline="") as file:
      writer = csv.writer(file)
      writer.writerow(["HostTaxID", "GenomeTaxID", "Sequence"])
      for genome in trainingGenomes:
          writer.writerow(genome)

trainingGenomeData = pd.read_csv(genomeFilePath)
trainingGenomeData = list(trainingGenomeData.itertuples(index=False, name=None))

In [19]:
# Preprocessing of Training Data
if doPreprocessing:
    dataPreprocessor = DataPreprocessing.DataPreprocessing()
    featureDataframe = dataPreprocessor.calculateFeatureMatrix(trainingGenomeData,chosenFeatures)
    dataPreprocessor.saveFeatureMatrixAsCSV(featureDataframe, featureFilePath)

featureData = pd.read_csv(featureFilePath)

# Convert the flat feature list into right format for classifier TODO: Do it better (best case before csv)
featureData['FeaturesFlat'] = featureData['FeaturesFlat'].apply(ast.literal_eval)
featureData['FeaturesFlat'] = featureData['FeaturesFlat'].apply(lambda x: [float(i) for i in x])

In [27]:
# Training of the PU-Classifier

# Define Hosts (with their HostTaxID) for which a model should be build and create a untrained classifier for each host
hosts = [10090, 9534]
classifiers = {}
for host in hosts:
    classifiers[host] = ElkanotoSVCClassifier()

# Create dictionary of HostTaxIDs (as key) and empty lists (of feature lists for each datapoint) for each host (as value)
featuresPerHost = {}
for host in hosts:
    featuresPerHost[host] = []

# Extract features of virus sequences for each host in seperate list and convert them to a np array
for _, row in featureData.iterrows():
    host = row['HostTaxID']
    features = np.array(row['FeaturesFlat'])
    featuresPerHost[host].append(features)

for host in hosts:
    featuresPerHost[host] = np.array(featuresPerHost[host])

# Transform features (X) and host classes (y) into expected format of PU Classifier and run k-fold cross validation for each host
X = np.concatenate([featuresPerHost[host] for host in hosts])

def getYForHost(h):
    y = []
    for host in hosts:
        y += [1 if host == h else 0] * len(featuresPerHost[host])
    return np.array(y)

# TODO: WHere is the training? -> Where are classifiers saved? Are multiple classifier trained?

scores = {}
for host in hosts:
    # load the training data
    X = X
    y = getYForHost(host)

    # run k-fold cross validation
    cv = KFold(n_splits=3, shuffle=True)
    #cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=42)
    scores[host] = cross_validate(classifiers[host], X, y, cv=cv, scoring=['balanced_accuracy', 'recall', 'precision'])

In [28]:
# Evaluation of the Training Process

# print out the metrics we want to see using the scores
scores

{10090: {'fit_time': array([0.84717178, 0.59498596, 0.77458286]),
  'score_time': array([0.22276711, 0.24308205, 0.13884521]),
  'test_balanced_accuracy': array([0.5, 0.5, 0.5]),
  'test_recall': array([1., 1., 1.]),
  'test_precision': array([0.81318681, 0.91111111, 0.86666667])},
 9534: {'fit_time': array([0.68211389, 0.67319202, 0.47704697]),
  'score_time': array([0.21298718, 0.18862104, 0.15947104]),
  'test_balanced_accuracy': array([0.5, 0.5, 0.5]),
  'test_recall': array([1., 1., 1.]),
  'test_precision': array([0.15384615, 0.12222222, 0.13333333])}}

In [ ]:
# Visualization of the results